In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='07-sklearn.ensemble.b-boosting'
classifierName='5-AdaBoostClassifier'
extraParameterName='1-Default'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Results

estimator__max_depthFound=1
# learning too low makes the process very slow
learning_rateFound=0.1
n_estimatorsFound=370

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,estimator__max_depthFound,n_estimatorsFound)


print('done')

In [ ]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
%%script false
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx0)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = AdaBoostClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.b-boosting','AdaBoostClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

# default result
# f1 train 0.0220
# f1 test : 0.0360

# Hyperparameters tuning


In [ ]:
%%script false

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = AdaBoostClassifier(random_state=42)
dic_param={
    'estimator': [DecisionTreeClassifier()],
    'estimator__max_depth':randint(1,9),
    'n_estimators': randint(35,90),
    'learning_rate': [0.1]
}
predictors = fd.getPredictors(dfTrx)
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None,iter=5)

print(res)

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 8, 'learning_rate': 0.1, 'n_estimators': 75}
#0.20380541883759157
#score   0.9888835186080231


In [ ]:
#%%script false
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

modelClf = AdaBoostClassifier(random_state=42)

 
dic_param={
     'estimator': [DecisionTreeClassifier()],
    'estimator__max_depth':[1],
    'n_estimators':[368,370,372],
    'learning_rate':[0.1]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 300}
#0.08770845920403302
#scoref1 0.08787632221318145

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 400}
#0.08947198501323246
#scoref1 0.0896

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 375}
#0.08953886244513908
#scoref1 0.088638195004029

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 370}
#0.08961914652727587
#scoref1 0.088638195004029





In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx0)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = AdaBoostClassifier(random_state=42)
parameters={'learning_rate': learning_rateFound, 'n_estimators':n_estimatorsFound, 'estimator':DecisionTreeClassifier(),
           'estimator__max_depth':estimator__max_depthFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result(packageName,classifierName,'2-After tuning', f1,mcc,roc)


f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
f1Test=fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07C-AdaBoostClassifierProbaHistogram' if saveImage else None)

diffF1=f1Train-f1Test
print("diffF1",diffF1)

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': modelClf.feature_importances_})
val=tmp[(tmp['Feature']=='amountBin')]['Feature importance']
amountImportance =val.values[0]
print("amount importance",amountImportance)


In [ ]:
files = fd.getAllFiles()
predictors = fd.getPredictors(dfTrx0)

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx=fd.getStandardScaledData(file,source,predictors)
    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3],diffF1,amountImportance)
